In [ ]:
import cadquery as cq
from jupyter_cadquery.cadquery import show, show_accuracy, show_constraints, Assembly, Part, Faces
from jupyter_cadquery import set_defaults, set_sidecar

# Avoid clean error
cq.occ_impl.shapes.Shape.clean = lambda x: x

set_sidecar("Bearing")
set_defaults(axes=False, axes0=True, edge_accuracy=0.01, mate_scale=1, zoom=3.5)

**Some helpers**

In [ ]:
def center(assy, name):
    return cq.Vertex.makeVertex(*assy.objects[name].obj.val().Center().toTuple())

def query_all(assy, q):
    name, kind, arg = q.split("@")

    tmp = cq.Workplane()
    obj = assy.objects[name].obj

    if isinstance(obj, (cq.Workplane, cq.Shape)):
        tmp.add(obj)
        res = getattr(tmp, kind)(arg)

    return res.objects if isinstance(res.val(), cq.Shape) else None

# Bearing

## Parts

In [ ]:
def ring(inner_radius, outer_radius, width):
    ring = (cq.Workplane(origin=(0, 0, -width / 2))
        .circle(outer_radius).circle(inner_radius)
        .extrude(width)
    )
    return ring

tol = 0.05
ball_diam = 5

r1, r2, r3, r4 = 4, 6, 8, 10
r5 = (r3 + r2) / 2
inner_ring = ring(r1, r2, ball_diam)
outer_ring = ring(r3, r4, ball_diam)

torus = cq.CQ(cq.Solid.makeTorus(r5, ball_diam / 2 + tol))
ball = cq.Workplane().sphere(ball_diam / 2)

inner = inner_ring.cut(torus)
outer = outer_ring.cut(torus)

show(Assembly([Part(ball, "ball"), Part(inner, "inner"), Part(outer, "outer")], "bearing"))

## Assembly

In [ ]:
def balls(i):
    return "ball_%d" % i

number_balls = 6

def create_bearing(cls, helpers=True):
    L = lambda *args: cq.Location(cq.Vector(*args))
    C = lambda *args: cq.Color(*args) 
    
    assy = cls(outer, loc=L(0, 0, ball_diam/2), name="outer", color=C("orange"))
    assy.add(inner, loc=L(20, 0, 0), name="inner", color=C("orange"))
    for i in range(number_balls):
        assy.add(ball, loc=L(6*i, 20, 0), name=balls(i), color=C("black"))

    if helpers:
        assy.add(cq.Workplane().circle(1).extrude(1), loc=L(0,-20,0), name="_center")
        assy.add(cq.Workplane().polygon(number_balls, 2*r5).extrude(ball_diam/2), loc=L(20,-20,0), name="_points")

    return assy

## Numerical solver from cadquery.Assembly

### Assembly

In [ ]:
set_defaults(zoom=3.5)

In [ ]:
bearing = create_bearing(cq.Assembly)
show(bearing)

### Constraints

In [ ]:
points = query_all(bearing, "_points@vertices@>Z")

cs = [
    ("outer@faces@<Z", "_center@faces@>Z", "Plane"),
    ("inner@faces@<Z", "_center@faces@>Z", "Plane"),
    ("_points@faces@<Z", "_center@faces@>Z", "Plane"),
] + [
    ("_points", points[i], balls(i), center(bearing, balls(i)), "Point") 
    for i in range(number_balls)
]

for c in cs:
    bearing.constrain(*c)
    
show_constraints(bearing, cs)

###  Solver

In [ ]:
bearing.solve()
show(bearing)

In [ ]:
show_accuracy(bearing, cs)

## Mate Assembly from jupyter-cadquery

### MAssembly

In [ ]:
from jupyter_cadquery.mate_assembly import Mate, MAssembly

bearing = create_bearing(MAssembly, helpers=False)
show(bearing)

### Mates

In [ ]:
M = lambda *args: Mate(bearing.find(*args))

bearing.mate(
    name="outer", selector="outer", origin=True, mate=M("outer", "faces@<Z"),
).mate(
    name="inner", selector="inner", origin=True,mate=M("inner", "faces@<Z")
)

for i in range(number_balls):
    bearing.mate(
        name=balls(i), selector=balls(i), origin=True, mate=Mate((0,0,0), (1,0,0), (0,0,1)),
    ).mate(
        name="inner_%d" %i, selector="inner", mate=Mate((0,0,0), (1,0,0), (0,0,1)).rz(i*60).tx(r5)
    )
show(bearing, render_mates=True)

### Relocate and assemble

In [ ]:
bearing.relocate()

bearing.assemble("inner", "outer")

for i in range(number_balls):
    bearing.assemble(balls(i), "inner_%d"%i)
    
show(bearing, render_mates=True)